In [2]:
import wandb
import pandas as pd

# WandB 프로젝트 불러오기
api = wandb.Api()
runs = api.runs("document-classification")

# 각 지표를 담을 리스트 초기화
run_data = []

for run in runs:
    # 각 run에서 필요한 지표를 추출
    run_data.append({
        "run_name": run.name,
        "train_loss": run.summary.get("final_train_loss"),
        "train_accuracy": run.summary.get("final_train_accuracy"),
        "train_f1": run.summary.get("final_train_f1"),
        "val_loss": run.summary.get("final_valid_loss"),
        "val_accuracy": run.summary.get("final_valid_accuracy"),
        "val_f1": run.summary.get("final_valid_f1"),
        "test_loss": run.summary.get("final_test_loss"),
        "test_accuracy": run.summary.get("final_test_accuracy"),
        "test_f1": run.summary.get("final_test_f1")
    })

# DataFrame으로 변환
df = pd.DataFrame(run_data)

# 각 지표의 중요도 설정 (가중치)
# 여기서는 단순히 모든 지표를 동일하게 고려
weights = {
    "train_loss": -1,
    "train_accuracy": 1,
    "train_f1": 1,
    "val_loss": -1,
    "val_accuracy": 1,
    "val_f1": 1,
    "test_loss": -1,
    "test_accuracy": 1,
    "test_f1": 1
}

# 가중치를 이용하여 종합 점수 계산
df["combined_score"] = (
    weights["train_loss"] * df["train_loss"] +
    weights["train_accuracy"] * df["train_accuracy"] +
    weights["train_f1"] * df["train_f1"] +
    weights["val_loss"] * df["val_loss"] +
    weights["val_accuracy"] * df["val_accuracy"] +
    weights["val_f1"] * df["val_f1"] +
    weights["test_loss"] * df["test_loss"] +
    weights["test_accuracy"] * df["test_accuracy"] +
    weights["test_f1"] * df["test_f1"]
)

# 종합 점수 기준 정렬
sorted_df = df.sort_values(by="combined_score", ascending=False)

# 상위 모델 선택
top_models = sorted_df.head(5)

# 결과 출력
print(top_models)

                 run_id  train_loss  train_accuracy  train_f1  val_loss  \
16  run-20240731-205219    0.030505        0.985441  0.985042  0.357655   
14  run-20240731-203631    0.029506        0.989991  0.987662  0.478369   
17  run-20240731-205452    0.053180        0.986351  0.984659  0.348565   
18  run-20240731-205954    0.053164        0.986351  0.984659  0.348585   
15  run-20240731-204356    0.019249        0.992721  0.992474  0.543583   

    val_accuracy    val_f1  test_loss  test_accuracy   test_f1  combined_score  
16      0.893617  0.891233   0.494570       0.894068  0.899300        4.665970  
14      0.897872  0.892138   0.442351       0.906780  0.911045        4.635262  
17      0.880851  0.848699   0.425285       0.860169  0.855943        4.589642  
18      0.880851  0.848699   0.425330       0.860169  0.855943        4.589593  
15      0.889362  0.881951   0.569090       0.906780  0.913922        4.445287  


In [2]:
import wandb
import pandas as pd
import numpy as np

# WandB 프로젝트 불러오기
api = wandb.Api()
runs = api.runs("document-classification")

# 각 지표를 담을 리스트 초기화
run_data = []

for run in runs:
    # 각 run에서 필요한 지표를 추출
    run_data.append({
        "run_name": run.name,
        "train_loss": run.summary.get("final_train_loss"),
        "train_accuracy": run.summary.get("final_train_accuracy"),
        "train_f1": run.summary.get("final_train_f1"),
        "val_loss": run.summary.get("final_valid_loss"),
        "val_accuracy": run.summary.get("final_valid_accuracy"),
        "val_f1": run.summary.get("final_valid_f1"),
        "test_loss": run.summary.get("final_test_loss"),
        "test_accuracy": run.summary.get("final_test_accuracy"),
        "test_f1": run.summary.get("final_test_f1")
    })

# DataFrame으로 변환
df = pd.DataFrame(run_data)

def calculate_updated_score(row):
    # 평가 세트(valid + test) 성능 계산
    eval_f1 = (row['val_f1'] + row['test_f1']) / 2
    eval_accuracy = (row['val_accuracy'] + row['test_accuracy']) / 2
    eval_loss = (row['val_loss'] + row['test_loss']) / 2

    # 성능 점수 계산
    performance_score = 0.5 * eval_f1 + 0.3 * eval_accuracy - 0.2 * eval_loss

    # 과적합 점수 계산
    overfitting_score = (row['train_accuracy'] - eval_accuracy) + (row['train_f1'] - eval_f1)
    overfitting_penalty = 1 / (1 + np.exp(-5 * (overfitting_score - 0.1)))  # 시그모이드 함수

    # 일관성 점수 계산 (valid와 test 간의 차이, 작을수록 좋음)
    consistency_score = 1 / (1 + abs(row['val_f1'] - row['test_f1']) + abs(row['val_accuracy'] - row['test_accuracy']))

    # 최종 점수 계산
    final_score = performance_score * (1 - 0.3 * overfitting_penalty) * consistency_score

    return final_score

# 종합 점수 계산 및 정렬
df['combined_score'] = df.apply(calculate_updated_score, axis=1)
sorted_df = df.sort_values(by="combined_score", ascending=False)

# 상위 모델 선택
top_models = sorted_df.head(5)

# 결과 출력
print(top_models)

               run_name  train_loss  train_accuracy  train_f1  val_loss  \
16  run-20240731-205219    0.030505        0.985441  0.985042  0.357655   
14  run-20240731-203631    0.029506        0.989991  0.987662  0.478369   
17  run-20240731-205452    0.053180        0.986351  0.984659  0.348565   
18  run-20240731-205954    0.053164        0.986351  0.984659  0.348585   
15  run-20240731-204356    0.019249        0.992721  0.992474  0.543583   

    val_accuracy    val_f1  test_loss  test_accuracy   test_f1  combined_score  
16      0.893617  0.891233   0.494570       0.894068  0.899300        0.512632  
14      0.897872  0.892138   0.442351       0.906780  0.911045        0.503786  
17      0.880851  0.848699   0.425285       0.860169  0.855943        0.472813  
18      0.880851  0.848699   0.425330       0.860169  0.855943        0.472808  
15      0.889362  0.881951   0.569090       0.906780  0.913922        0.472731  
